In [ ]:
#download the module needed for this training

!pip install gdown
!pip install ultralytics
!pip install albumentations

In [ ]:
#yaml

!gdown 1KdCvJ-l53ZLbdt4xfw5G6h8Yq33I6IQK

In [ ]:
#dataset black

!gdown 1Hpqf1bTODohBaisjE1RAWwhuy_VNM3fm

In [ ]:
#dataset white

!gdown 1N0CetN7qs-NKCNZCFSxgICv-1D98BeIo

In [ ]:
# remove the datasets directory, just in case it contains other dataset
# make a directory for the dataset

!rm -rf datasets
!mkdir datasets
!mkdir datasets/data

In [ ]:
# extract black background

!tar -xf data.tar.xz -C datasets/data

In [ ]:
# extract white background

!tar -xf dataset-white.tar.xz -C datasets/data

In [ ]:
# delete augmented

!rm -rf /kaggle/working/datasets/data/train/*-aug.png
!rm -rf /kaggle/working/datasets/data/train/*-aug.jpg
!rm -rf /kaggle/working/datasets/data/train/*-aug.txt

In [ ]:
# run this if you are training "Researchers' Custom Augmentation"

def yolo_bboxes_txt_to_list_bboxes(file):
    with open(file, "r") as f:
        lines = f.readlines()
    lines = [line.strip().split() for line in lines]
    lines = [[float(x) for x in line] for line in lines]

    for i in range(len(lines)):
        lines[i] = lines[i][1:] + [lines[i][0]]

    return lines

def list_bboxes_to_yolo_bboxes_txt(bboxes, file_name):
    with open(file_name, "w") as f:
        for bbox in bboxes:
            x_center, y_center, width, height, class_id = bbox
            class_id = int(class_id)
            bboxes = f"{class_id} {x_center} {y_center} {width} {height}\n"
            f.write(bboxes)

import glob
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from pathlib import Path

dataset_train_folder = "/kaggle/working/datasets/data/train"

for train_image in glob.glob(dataset_train_folder + "/*.jpg"):
    #print(train_image)
    file_name_without_extension = Path(train_image).stem

    image = cv2.imread(train_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    transform = A.Compose([
        A.HorizontalFlip(p=1),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=(-180, 180), p=0.5, border_mode=cv2.BORDER_CONSTANT),
        A.ColorJitter(p=0.5)
    ], bbox_params=A.BboxParams(format="yolo"))

    yolo_file_bboxes = yolo_bboxes_txt_to_list_bboxes(
        f"{dataset_train_folder}/{file_name_without_extension}.txt")
    try:
        transformed = transform(image=image, bboxes=yolo_file_bboxes)
    except ValueError as e:
        print(train_image)
        print(e)

    transformed_image = transformed["image"]
    transformed_bbox = transformed["bboxes"]

    cv2.imwrite(f"{dataset_train_folder}/{file_name_without_extension}-aug.jpg", transformed_image)
    list_bboxes_to_yolo_bboxes_txt(transformed_bbox, f"{dataset_train_folder}/{file_name_without_extension}-aug.txt")

for train_image in glob.glob(dataset_train_folder + "/*.png"):
    file_name_without_extension = Path(train_image).stem

    image = cv2.imread(train_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    transform = A.Compose([
        A.HorizontalFlip(p=1),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=(-180, 180), p=0.5, border_mode=cv2.BORDER_CONSTANT),
        A.ColorJitter(p=0.5)
    ], bbox_params=A.BboxParams(format="yolo"))

    yolo_file_bboxes = yolo_bboxes_txt_to_list_bboxes(
        f"{dataset_train_folder}/{file_name_without_extension}.txt")

    try:
        transformed = transform(image=image, bboxes=yolo_file_bboxes)
    except ValueError as e:
        print(train_image)
        print(e)

    transformed_image = transformed["image"]
    transformed_bbox = transformed["bboxes"]

    cv2.imwrite(f"{dataset_train_folder}/{file_name_without_extension}-aug.png", transformed_image)
    list_bboxes_to_yolo_bboxes_txt(transformed_bbox, f"{dataset_train_folder}/{file_name_without_extension}-aug.txt")

print("Success")

In [ ]:
# delete cache if u changed ur dataset

!rm -rf /kaggle/working/datasets/data/train/train.cache

In [ ]:
# install this before training

pip install -U ipywidgets

In [ ]:
# install this for visualization of results

!pip install wandb

In [ ]:
wandb login

In [ ]:
# 1st Experiment: Without Augmentation

from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data='fishscan.yaml',
    epochs=200,
    imgsz=320,
    optimizer='AdamW',
    lr0=0.001,
    patience=10,
    device=[0,1],
    # augmentations
    hsv_h=0.0,hsv_s=0.0,hsv_v=0.0,
    translate=0.0,scale=0,mosaic=0,erasing=0,
    degrees=0,
    flipud=0.0,
    fliplr=0.0,
    copy_paste=0.5,
    project="<name of your project>", # eg. black-without-aug
    plots=True,
    augment=False,
)

In [ ]:
# 2nd Experiment: YOLOv8 Default Augmentation

from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data='fishscan.yaml',
    epochs=200,
    imgsz=320,
    optimizer='AdamW',
    lr0=0.001,
    patience=10,
    device=[0,1],
    project="<name of the project>", # eg. black-yolov8-def
    plots=True,
    augment=True,
)

In [ ]:
# 3rd Experiment: Researchers' Custom Augmentation

from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data='fishscan.yaml',
    epochs=200,
    imgsz=640,
    optimizer='AdamW',
    lr0=0.001,
    patience=10,
    device=[0,1],
    hsv_h=0.0,hsv_s=0.0,hsv_v=0.0,
    translate=0.0,scale=0,mosaic=0,erasing=0,
    degrees=0,
    flipud=0.5,
    fliplr=0.5,
    copy_paste=0.0,
    project="<name of the project>", # eg. black-res-custom
    plots=True,
    augment=True,
)

In [ ]:
# 4th Experiment: Combination of YOLOv8 Default Augmentation
# and Researchers' Custom Augmentation

from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data='fishscan.yaml',
    epochs=200,
    imgsz=320,
    optimizer='AdamW',
    lr0=0.001,
    patience=10,
    device=[0,1],
    flipud=0.5,
    fliplr=0.5,
    project="<name of your project>", # eg. black-combi
    plots=True,
    augment=True,
)

In [ ]:
# save your training after 200 epochs or after patience

!zip /kaggle/working/<name of the zipfile>.zip -r /kaggle/working/combined-our-aug-640/train3

In [ ]:
# test results

from ultralytics import YOLO
model = YOLO("/kaggle/working/combined-our-aug-640/train2/weights/best.pt") # directory of best.pt
results = model.val(data='fishscan.yaml',save=True,plots=True, split='test')

In [ ]:
#save the tests results

!zip /kaggle/working/<name of the zipfile>.zip -r /kaggle/working/runs/detect/val